In [1]:
import pandas as pd

/Users/aldemarogonzalez/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/2r/b60vp1dj2zn0rg5yg_gyy0zr0000gq/T/ipykernel_3435/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# organization_path = '../Dataset/bulk_export/organizations.csv' 
organization_path = '../Dataset/filtered_data/organizations_filtered_by_date_and_roles.csv' 
organization_data = pd.read_csv(organization_path)

# Step 2: Manipulate or analyze the data as needed
# For example, you can print the first few rows of the DataFrame using .head()
# print("First 5 rows of the DataFrame:")
# print(organization_data.head())

# Or you can perform some basic analysis, such as getting summary statistics using .describe()
# print("\nSummary statistics of the DataFrame:")
# print(organization_data.describe())

organization_data_num_rows = organization_data.shape[0]
print("Number of Organization in the Organization.csv:", organization_data_num_rows)
organization_data_num_columns = organization_data.shape[1] 
print("Number of columns in the Organization.csv:", organization_data_num_columns)
organization_data_column_names = organization_data.columns
print("Column names in the Organization.csv:")
for name in organization_data_column_names:
    print(name)

In [ ]:
people_path = '../Dataset/bulk_export/people.csv'  # Replace 'your_file.csv' with the actual path to your CSV file
people_data = pd.read_csv(people_path)
people_data_num_rows = people_data.shape[0] 
print("Number of People in the People.csv:", people_data_num_rows)
people_data_num_columns = people_data.shape[1] 
print("Number of columns in the People.csv:", people_data_num_columns)
people_data_column_names = people_data.columns
print("Column names in the People.csv:")
for name in people_data_column_names:
    print(name)

In [ ]:
funding_path = '../Dataset/bulk_export/funding_rounds.csv'
funding_data = pd.read_csv(funding_path)
funding_data_num_rows = funding_data.shape[0] 
print("Number of funding in the funding.csv:", funding_data_num_rows)
funding_data_num_columns = funding_data.shape[1]
print("Number of columns in the funding.csv:", funding_data_num_columns)
funding_data_column_names = funding_data.columns
print("Column names in the funding.csv:")
for name in funding_data_column_names:
    print(name)

# Filtering the companies

1 - I filtered companies using their creation date

In [14]:
from datetime import datetime

# Define the warm up window 
start_date = datetime.strptime('2015-01-01', '%Y-%m-%d') # Start Day: 1 of January of 2015
end_date = datetime.strptime('2018-12-31', '%Y-%m-%d') # End Day: 31 of December of 2018

In [ ]:
import csv

with open('../Dataset/bulk_export/organizations.csv', 'r') as f, open('../Dataset/filtered_data/organizations_filtered_by_date.csv', 'w', newline='') as f_out:
    reader = csv.reader(f)
    writer = csv.writer(f_out)

    header = next(reader)
    writer.writerow(header)

    date_index = header.index('founded_on')

    # Read the rest of the csv and write rows where the date is within the range
    for row in reader:
        date_str = row[date_index]
        if date_str:  # Check if the date string is not empty
            try:
                date = datetime.strptime(date_str, '%Y-%m-%d')
                if start_date <= date <= end_date:
                    writer.writerow(row)
            except ValueError:
                print("Error: Unable to parse date string '{}'".format(date_str))
        else:
            print("Warning: Empty date string encountered in row {}".format(row))

Looking at the dataset, we see that there is a column that also let us differentiate between an Companies and Investment firms in the domain column

In [ ]:
with open('../Dataset/filtered_data/organizations_filtered_by_date.csv', 'r') as f, open('../Dataset/filtered_data/organizations_filtered_by_date_and_roles.csv', 'w', newline='') as f_out:
    reader = csv.reader(f)
    writer = csv.writer(f_out)

    header = next(reader)  # This gets the header row
    writer.writerow(header)  # Write the header to the new file

    # Find the index of the date column (replace 'date_column' with your column's name)
    roles_index = header.index('roles')

    # Read the rest of the csv and write rows where the date is within the range
    for row in reader:
        roles_string = row[roles_index]
        if roles_string:  # Check if the date string is not empty
            try:
                if roles_string == "company":
                    writer.writerow(row)
            except ValueError:
                print("Error: Unable to parse string string '{}'".format(roles_string))
        else:
            print("Warning: Empty string string encountered in row {}".format(row))


Clean the database by dropping the columns that are not relevant for the study

In [ ]:
# Specify the columns to drop
columns_to_drop = ['type', 
    'cb_url', 
    'rank', 
    'created_at', 
    'updated_at', 
    'legal_name', 
    'roles', 
    'cb_url', 
    'rank', 
    'created_at', 
    'updated_at', 
    'legal_name', 
    'roles', 
    'email', 
    'phone', 
    'logo_url', 
    'alias1', 
    'alias2', 
    'alias3', 
    'primary_role', 
    'num_exits']

# Drop the specified columns
organizations_data = organization_data.drop(columns=columns_to_drop)
organizations_data.to_csv('../Dataset/filtered_data/filtered_organization.csv', index=False)

### Now I merge the organizations with their funding data to further filtering by Venture Stage/Acquisition/Closure

In [5]:
funding_path = '../Dataset/bulk_export/funding_rounds.csv' 

funding_data = pd.read_csv(funding_path)

In [7]:
organizations_path = '../Dataset/filtered_data/filtered_organization.csv'

organizations_data = pd.read_csv(organizations_path)

In [9]:
merged_df = organizations_data.merge(funding_data, left_on='uuid', right_on='org_uuid', how="left")

In [11]:
merged_df.to_csv('../Dataset/filtered_data/filtered_organization_merged.csv', index=False)

First - I removed the ones closed during the Warmup window 

In [18]:
merged_df['closed_on'] = pd.to_datetime(merged_df['closed_on'])
filtered_close_merged_df = merged_df[(merged_df['closed_on'] < start_date) | (merged_df['closed_on'] > end_date) | (merged_df['closed_on'].isna())]


Now I removed the ones acquired during the Warmup window 

In [19]:
acquisitions_path = '../Dataset/bulk_export/acquisitions.csv'  # Replace 'your_file.csv' with the actual path to your CSV file
acquisitions_data = pd.read_csv(acquisitions_path)
acquisitions_data_num_rows = acquisitions_data.shape[0] 
print("Number of acquisitions in the acquisitions.csv:", acquisitions_data_num_rows)
acquisitions_data_num_columns = acquisitions_data.shape[1] 
print("Number of columns in the acquisitions.csv:", acquisitions_data_num_columns)
acquisitions_data_column_names = acquisitions_data.columns
print("Column names in the acquisitions.csv:")
for name in acquisitions_data_column_names:
    print(name)

Number of acquisitions in the acquisitions.csv: 162216
Number of columns in the acquisitions.csv: 27
Column names in the acquisitions.csv:
uuid
name
type
permalink
cb_url
rank
created_at
updated_at
acquiree_uuid
acquiree_name
acquiree_cb_url
acquiree_country_code
acquiree_state_code
acquiree_region
acquiree_city
acquirer_uuid
acquirer_name
acquirer_cb_url
acquirer_country_code
acquirer_state_code
acquirer_region
acquirer_city
acquisition_type
acquired_on
price_usd
price
price_currency_code


In [21]:
acquired_df =  filtered_close_merged_df.merge(acquisitions_data, left_on='uuid_x', right_on='acquiree_uuid', how="left")

In [26]:
acquired_df.to_csv('../Dataset/filtered_data/acquired_df.csv', index=False)

In [23]:
merged_df['acquired_on'] = pd.to_datetime(merged_df['acquired_on'])
filtered_close_merged_df = merged_df[(merged_df['acquired_on'] < start_date) | (merged_df['acquired_on'] > end_date) | (merged_df['acquired_on'].isna())]


Now I removed the ones that IPO during the Warmup window 

Second - I filter the companies which closed a funding round above Series B or above, which is not interesting for Plug and Play investments


In [ ]:
with open('../Dataset/filtered_data/organizations_filtered_by_date.csv', 'r') as f, open('../Dataset/filtered_data/organizations_filtered_by_funding.csv', 'w', newline='') as f_out:
    reader = csv.reader(f)
    writer = csv.writer(f_out)

    header = next(reader)  # This gets the header row
    writer.writerow(header)  # Write the header to the new file

    # Find the index of the date column (replace 'date_column' with your column's name)
    date_index = header.index('round_type')

    series_b_or_above = ['series_b', 'series_c', 'series_d', 'series_e', 'series_f', 'series_g']

    # Initialize a counter for the number of rows written
    rows_written = 0

    # Read the rest of the csv and write rows where the date is within the range
    for row in reader:
        series_str = row[date_index]
        if series_str:  # Check if the date string is not empty
            if series_str not in series_b_or_above:
                writer.writerow(row)
                rows_written += 1
                if rows_written % 100 == 0:  # Check if 100 rows have been written
                    print(f"{rows_written} rows written")
                if rows_written == 100:  # Stop after 100 rows
                    break
        else:
            print("Warning: Empty date string encountered in row {}".format(row))

-------------

Filter Out Series B or Above
After merging, filter out organizations that have raised a Series B or above. Assuming the funding round information is stored in a column named round_type in the funding_rounds DataFrame, you can do this by excluding rows where round_type is in a list of Series B or above.